In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.io as sio
import os

from multiprocessing import Pool
from tqdm import tqdm
from teaspoon.parameter_selection.FNN_n import FNN_n
from teaspoon.parameter_selection.MI_delay import MI_for_delay

from functions import init_Frame, SSub_ntau, twodim_graphs, check_maxmin, tuplinator

#Not used for now
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

**Backward Masking Experiment Dictionary**

In [ ]:
#Conditions codes
conditions_dict = {'con_left_self': 'S__1',
                   'con_left_other': 'S__2',
                   'con_right_self': 'S__3',
                   'con_right_other': 'S__4',
                   'uncon_left_self': 'S_11',
                   'uncon_left_other': 'S_12',
                   'uncon_right_self': 'S_13',
                   'uncon_right_other': 'S_14',}

sub_list = ['001','002','003','004','005','006','007','008','009','010',
            '011','012','013','014','015','016','017','018','019','020',
            '023','024','025','026','028','029','030',
            '031','033','034','035','036','037','038','040',
            '042']

**Get elecrodes indexing and select points of interest**

In [ ]:
#Data paths

#Workflow folder path
path = '/home/lunis/Documents/nlin-EEG/'

#Backward Masking dataset path
bw_path = path + 'data/backward_masking/'

#Load channel names
path = bw_path + 'subj001_band_resample/channel.mat'
#ch_clust = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'T7', 'T8', 'FT7',
#             'FT8', 'O1', 'O2', 'PO7', 'PO8', 'P5', 'P6', 'P7', 'P8']
#ch_clust = ['P5', 'P7', 'PO8', 'P6', 'P8']
ch_clust = ['P5', 'P7', 'P6'], ['P6', 'P8']
#ch_clust = ['P5', 'P7', 'P6'], ['P5', 'P7', 'P6']
#ch_clust = ['P5']

#Make a tuple out of a list
#ch_clust = tuplinator(ch_clust)

# Load the .mat file (adjust the path)
mat_data = sio.loadmat(path)

ch_list = []
for m in mat_data['Channel'][0]:
#     print(m[0][0])
    ch_list.append(str(m[0][0]))

if type(ch_clust) ==  tuple:

    first = True
    for c in ch_clust:
        part = []
        
        for sc in c:
        
            part = part + [np.where(np.asarray(ch_list)==sc)[0][0]]

        part = part,
        if first == True:

            ch_clust_idx = part
            first = False

        else:
            ch_clust_idx = ch_clust_idx + part
            

else:

    ch_clust_idx = []
    for c in ch_clust:
        ch_clust_idx.append(np.where(np.asarray(ch_list)==c)[0][0])


**Print 2-dim Time Embedding trajectories**

In [ ]:
###Set parameters

#Time delay
tau = 20

#Number of datapoints to ignore from start and end of signals
trim = 0

#Experiment conditions to pull from
conditions = ['S__', 'S_1']

#POIs (Put a function here at some point)
poi_idx = ch_clust_idx

#Initialize directory
os.makedirs('./pics/backward_masking/2dim_emb/', exist_ok = True)

###Multiprocessing

#Create iterable function
def Sub_2dimPics(subID):

    twodim_graphs(subID, tau = tau, trim = trim, conditions = conditions, channels_idx = poi_idx)

    return

#Set multiprocessing parameters
workers = os.cpu_count()-4
chk_size = int(len(sub_list)/workers) + 1

#Run calculations
if __name__ == '__main__':
    with Pool(workers) as p:
        p.map(Sub_2dimPics, sub_list, chk_size)


**Calculate and save optimal $\tau$ and $n$ for Time Embedding**

In [ ]:
###Set parameters

#Metrics calculated by SSub_ntau
metrics = ['t', 'n']

#Experiment conditions to pull from
conditions = ['S__', 'S_1']

#POIs (Put a function here at some point)
poi_idx = ch_clust_idx

#Initzialize dataframe for results
df = init_Frame(conditions, poi_idx, metrics)

###Multiprocess

#Create iterable function
def Sub_Results(subID):

    rs = SSub_ntau(subID, conditions = conditions, channels_idx = poi_idx)

    return rs

#Set multiprocessing parameters
workers = os.cpu_count()-4
chk_size = int(len(sub_list)/workers) + 1

#Run calculations
if __name__ == '__main__':
    with Pool(workers) as p:
        rows = p.imap(Sub_Results, sub_list, chk_size)

        i = 0
        for row in rows:

            df.loc[i] = row

            i = i +1

###Save results
df.to_csv(path + 'embedding.csv', sep = ';', index = False)
df.head()

**Plot time signal of POIs**

In [ ]:
#Plotting and gathering files in a smart way (Courtesy of Alessio)

conds = ['S__', 'S_1'] # S__ : conscious, S_1 : unconscious
fig,axs = plt.subplots(6,6, figsize=(12,10))
for i, ax in enumerate(axs.flat):
    print('sub '+str(i))
    sub = sub_list[i]
    folder = '/home/lunis/Documents/EEG/data/backward_masking/subj' + sub + '_band_resample/'
    all_files = os.listdir(folder)
    # cond = 'S_1' # unconscious

    # Loop over conditions (conscious and unconscious)
    for cond in conds:
        my_cond_files = [f for f in all_files if cond in f ]

        # Loop over trials of the present conditions
        all_trials = np.empty((0,451))
        for f in my_cond_files:
            path = folder+f
            # Load the .mat file (adjust the path)
            mat_data = sio.loadmat(path)

            # Inspect the keys in the .mat file
    #         print(mat_data['F'][ch_clust_idx].shape)
            data = mat_data['F'][ch_clust_idx]
    #         plt.plot(data.mean(axis=0))
        #     for e in data:
        #         plt.plot(e)
            all_trials = np.concatenate((all_trials, data.mean(axis=0)[np.newaxis, :]))
#         print(all_trials.shape)
        ax.plot(all_trials.mean(axis=0), label=cond)
        ax.axvspan(150, 250, color='grey', alpha=0.15)
#     ax.legend()
plt.savefig(path + 'plottini_all_sub_18_el.png', dpi=300)

**Pyplot testing cell**

In [ ]:
st = np.array([1,3,7,3,5,8,3])

emb = np.empty((2, len(st)- 1))

for i in range(0, len(st) - 1):

    emb[0][i] = st[i]
    emb[1][i] = st[i + 1]

fig, ax = plt.subplots()
ax.plot(emb[0,:], emb[1,:], 'ko-', linewidth = 1, markersize = 3)

ax.set(title = 'AAAaaaAaA')

**Check for weird stuff**

In [ ]:
###Set parameters

#Metrics calculated by check_maxmin
metrics = ['max', 'min']

#Experiment conditions to pull from
conditions = ['S__', 'S_1']

#POIs (Put a function here at some point)
poi_idx = ch_clust_idx

#Initzialize dataframe for results
df = init_Frame(conditions, poi_idx, metrics)

###Multiprocess

#Create iterable function
def Sub_Results(subID):

    rs = check_maxmin(subID, conditions = conditions, channels_idx = poi_idx)

    return rs

#Set multiprocessing parameters
workers = os.cpu_count()-4
chk_size = int(len(sub_list)/workers) + 1

#Run calculations
if __name__ == '__main__':
    with Pool(workers) as p:
        rows = p.imap(Sub_Results, sub_list, chk_size)

        i = 0
        for row in rows:

            df.loc[i] = row

            i = i +1

###Save results
df.to_csv(path + 'maxmin.csv', index = False)
df.head()